In [1]:
import pandas as pd

In [2]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [3]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,1670
2,Huelva,Confirmados PDIA 14 días,505
3,Huelva,Tasa PDIA 14 días,"98,87555972377497"
4,Huelva,Confirmados PDIA 7 días,233
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,2
629,Municipio de Huelva sin especificar,Total Confirmados,25
630,Municipio de Huelva sin especificar,Curados,1


In [4]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [5]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [6]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  1670.0


In [7]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  402.0


In [8]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [9]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 3057 personas en los últimos 7 días 

Un positivo PCR cada 1528 personas en los últimos 14 días


In [10]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [11]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [12]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [13]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PCR']

In [14]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,1670.0,233.0,505.0,510743.0,45.619813,98.875560,150.0
Condado-Campiña,590.0,123.0,238.0,155057.0,79.325667,153.491942,84.0
Huelva-Costa,832.0,93.0,209.0,288115.0,32.278778,72.540479,56.0
San Juan del Puerto,58.0,37.0,37.0,9300.0,397.849462,397.849462,34.0
Huelva (capital),402.0,47.0,94.0,143663.0,32.715452,65.430904,34.0
Bollullos Par del Condado,105.0,21.0,59.0,14272.0,147.141256,413.396861,16.0
Lepe,122.0,26.0,48.0,27431.0,94.783274,174.984507,11.0
Sierra de Huelva-Andévalo Central,223.0,15.0,54.0,67571.0,22.198872,79.915940,10.0
Palma del Condado (La),82.0,19.0,46.0,10761.0,176.563516,427.469566,10.0


In [15]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [16]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Manzanilla,19.0,7.0,15.0,2135.0,327.868852,702.576112,0.0
Almendro (El),5.0,1.0,4.0,826.0,121.065375,484.261501,0.0
Valdelarco,2.0,1.0,1.0,239.0,418.410042,418.410042,0.0
Palma del Condado (La),81.0,27.0,45.0,10761.0,250.906050,418.176749,9.0
Bollullos Par del Condado,103.0,23.0,57.0,14272.0,161.154709,399.383408,14.0
San Juan del Puerto,50.0,29.0,31.0,9300.0,311.827957,333.333333,26.0
Villarrasa,10.0,6.0,7.0,2176.0,275.735294,321.691176,2.0
Villanueva de los Castillejos,10.0,7.0,8.0,2820.0,248.226950,283.687943,3.0
Lucena del Puerto,12.0,5.0,7.0,3371.0,148.323939,207.653515,3.0
